In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = de5d43e
molecule = {'h2o': 'atmpro', 'co2': 0.0004}
band = [11]
atmpro = trp
tsfc = 300


...

Best-fit values for each (gas, band)
------------------------------------
o h2o band9
  atmpro = mls
  band = 9
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 1e-24
  molecule = h2o
  ng_adju = [0]
  ng_refs = [5]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(500, 250)]
  tsfc = 294
  vmax = 3000
  vmin = 1900
  w_diffuse = [(1.66, 1.66, 1.66, 1.66, 1.66)]
  wgt = [(0.4, 0.4, 0.5, 0.55, 0.85)]
o co2 band9
  atmpro = mls
  band = 9
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.5e-24
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [3, 3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (50, 250)]
  tsfc = 294
  vmax = 3000
  vmin = 1900
  w_diffuse = [(1.66, 1.66, 1.75), (1.75, 1.6, 1.85)]
  wgt = [(0.7, 0.8, 0.7), (0.8, 0.7, 0.8)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0  0.135159                  NaN       NaN
0.000312      NaN       NaN                  1.0  0.266145
0.000750      2.0  0.010779                  2.0  0.084550
0.001052      3.0  0.008704                  3.0  0.045136
0.001476      4.0  0.003673                  4.0  0.008407
0.002070      5.0 -0.001149                  5.0 -0.018632
0.002904      6.0 -0.005399                  6.0 -0.033793
0.004074      7.0 -0.009281                  7.0 -0.040319
0.005714      8.0 -0.013051                  8.0 -0.041657
0.008015      9.0 -0.016945                  9.0 -0.042178
0.011243     10.0 -0.021130                 10.0 -0.042404
0.015771     11.0 -0.025744                 11.0 -0.043541
0.022122     12.0 -0.031029                 12.0 -0.046076
0.031031     13.0 -0.038005                 13.0 -0.051088
0.043528     14.0 -0.050464                 14.0 -0.062958
0.061057     15.0 -0.058837                 15.0 -0.068705
0.085645     16.0 -0.056266                 16.0 -0.058806
0.120136     17.0 -0.049540                 17.0 -0.041004
0.168516     18.0 -0.040602                 18.0 -0.021367
0.236378     19.0 -0.024440                 19.0 -0.001838
0.331549     20.0  0.004690                 20.0  0.017472
0.465100     21.0  0.063415                 21.0  0.053351
0.652400     22.0  0.181212                 22.0  0.152362
0.915100     23.0  0.211797                 23.0  0.176033
1.283650     24.0  0.150577                 24.0  0.138557
1.800600     25.0  0.104316                 25.0  0.118033
2.525700     26.0  0.061807                 26.0  0.086466
3.542800     27.0  0.036705                 27.0  0.057165
4.969550     28.0  0.021989                 28.0  0.029245
6.970850     29.0  0.012397                 29.0  0.009575
9.778100     30.0  0.006108                 30.0  0.003614
13.715850    31.0  0.002197                 31.0  0.004677
19.239350    32.0  0.000065                 32.0  0.003569
26.987250    33.0 -0.000769                 33.0  0.000955
37.855300    34.0 -0.000773                 34.0 -0.000740
53.100050    35.0 -0.001389                 35.0 -0.001837
73.887500    36.0 -0.001965                 36.0 -0.002286
97.662500    37.0 -0.002285                 37.0 -0.002574
121.437500   38.0 -0.002522                 38.0 -0.002810
145.212500   39.0 -0.002494                 39.0 -0.002754
168.987500   40.0 -0.002667                 40.0 -0.003061
192.762500   41.0 -0.002831                 41.0 -0.003341
216.537500   42.0 -0.002984                 42.0 -0.003624
240.312500   43.0 -0.003087                 43.0 -0.003841
264.087500   44.0 -0.003135                 44.0 -0.004069
287.862500   45.0 -0.003119                 45.0 -0.004164
311.637500   46.0 -0.003024                 46.0 -0.003986
335.412500   47.0 -0.002839                 47.0 -0.003695
359.187500   48.0 -0.002565                 48.0 -0.003504
382.962500   49.0 -0.002165                 49.0 -0.003476
406.737500   50.0 -0.001548                 50.0 -0.003314
430.512500   51.0 -0.000720                 51.0 -0.002741
454.287500   52.0  0.000331                 52.0 -0.001586
478.062500   53.0  0.001612                 53.0  0.000095
501.837500   54.0  0.003127                 54.0  0.002364
525.612500   55.0  0.004893                 55.0  0.004833
549.387500   56.0  0.006905                 56.0  0.007478
573.162500   57.0  0.009191                 57.0  0.009983
596.937500   58.0  0.011814                 58.0  0.012253
620.712500   59.0  0.014860                 59.0  0.014388
644.487500   60.0  0.018542                 60.0  0.016778
668.262500   61.0  0.022985                 61.0  0.019797
692.037500   62.0  0.027717                 62.0  0.023279
715.812500   63.0  0.031291                 63.0  0.025897
739.587500   64.0  0.033872                 64.0  0.027776
76

# Fluxes by Layer

CRD                        CLIRAD                    
                        flug      fldg     fnetg      flug      fldg     fnetg
pressure     level                                                            
0.000000e+00 1     -4.705975  0.000000 -4.705975       NaN       NaN       NaN
1.000000e-08 1           NaN       NaN       NaN -4.592558  0.000002 -4.592556
6.244000e-04 2     -4.705980  0.000014 -4.705965 -4.592565  0.000029 -4.592536
8.759000e-04 3     -4.705982  0.000017 -4.705965 -4.592568  0.000035 -4.592534
1.228600e-03 4     -4.705985  0.000020 -4.705964 -4.592573  0.000041 -4.592532
1.723400e-03 5     -4.705988  0.000024 -4.705964 -4.592578  0.000047 -4.592531
2.417400e-03 6     -4.705993  0.000029 -4.705964 -4.592586  0.000053 -4.592533
3.390900e-03 7     -4.705999  0.000034 -4.705965 -4.592595  0.000059 -4.592536
4.756500e-03 8     -4.706008  0.000041 -4.705966 -4.592608  0.000065 -4.592543
6.672000e-03 9     -4.706020  0.000050 -4.705970 -4.592626  0.000074 -4.592553
9.358900e-03 10    -4.706036  0.000061 -4.705975 -4.592651  0.000085 -4.592566
1.312780e-02 11    -4.706058  0.000074 -4.705984 -4.592685  0.000100 -4.592585
1.841450e-02 12    -4.706090  0.000090 -4.706000 -4.592733  0.000121 -4.592612
2.583020e-02 13    -4.706136  0.000108 -4.706028 -4.592802  0.000149 -4.592653
3.623230e-02 14    -4.706205  0.000130 -4.706075 -4.592903  0.000188 -4.592715
5.082340e-02 15    -4.706319  0.000158 -4.706162 -4.593065  0.000241 -4.592824
7.129060e-02 16    -4.706514  0.000210 -4.706304 -4.593324  0.000333 -4.592991
1.000000e-01 17    -4.706825  0.000329 -4.706496 -4.593712  0.000521 -4.593191
1.402710e-01 18    -4.707301  0.000569 -4.706732 -4.594274  0.000888 -4.593387
1.967600e-01 19    -4.708017  0.001013 -4.707004 -4.595077  0.001547 -4.593530
2.759970e-01 20    -4.709059  0.001826 -4.707233 -4.596202  0.002655 -4.593547
3.871000e-01 21    -4.710485  0.003313 -4.707171 -4.597715  0.004398 -4.593317
5.431000e-01 22    -4.712031  0.006031 -4.706000 -4.599401  0.007070 -4.592331
7.617000e-01 23    -4.711969  0.010662 -4.701307 -4.599388  0.011003 -4.588385
1.068500e+00 24    -4.709403  0.015793 -4.693610 -4.596869  0.014882 -4.581987
1.498800e+00 25    -4.705504  0.019570 -4.685935 -4.593102  0.018178 -4.574924
2.102400e+00 26    -4.700696  0.022220 -4.678476 -4.588439  0.021955 -4.566483
2.949000e+00 27    -4.695833  0.023556 -4.672278 -4.583577  0.025766 -4.557811
4.136600e+00 28    -4.691184  0.024070 -4.667114 -4.578669  0.028901 -4.549768
5.802500e+00 29    -4.686774  0.023999 -4.662775 -4.573763  0.029767 -4.543997
8.139200e+00 30    -4.682657  0.023313 -4.659343 -4.568881  0.027535 -4.541346
1.141700e+01 31    -4.678913  0.021942 -4.656971 -4.564186  0.024244 -4.539943
1.601470e+01 32    -4.675650  0.019876 -4.655775 -4.559907  0.022512 -4.537395
2.246400e+01 33    -4.672990  0.017266 -4.655725 -4.556240  0.021571 -4.534669
3.151050e+01 34    -4.671022  0.014473 -4.656548 -4.553192  0.019547 -4.533645
4.420010e+01 35    -4.669551  0.011840 -4.657711 -4.550397  0.015641 -4.534757
6.200000e+01 36    -4.669585  0.008946 -4.660639 -4.549136  0.010507 -4.538630
8.577500e+01 37    -4.672487  0.006314 -4.666174 -4.551665  0.006596 -4.545069
1.095500e+02 38    -4.677700  0.005092 -4.672608 -4.557551  0.005233 -4.552318
1.333250e+02 39    -4.685295  0.005586 -4.679710 -4.566601  0.006369 -4.560232
1.571000e+02 40    -4.694800  0.008066 -4.686734 -4.577810  0.009821 -4.567989
1.808750e+02 41    -4.706323  0.012077 -4.694246 -4.591215  0.014604 -4.576611
2.046500e+02 42    -4.720047  0.017828 -4.702219 -4.606954  0.020931 -4.586023
2.284250e+02 43    -4.736424  0.025800 -4.710623 -4.625489  0.029259 -4.596230
2.522000e+02 44    -4.755864  0.036547 -4.719317 -4.647137  0.040089 -4.607048
2.759750e+02 45    -4.778882  0.050737 -4.728145 -4.672405  0.053896 -4.618509
2.997500e+02 46    -4.806089  0.069160 -4.736930 -4.701871  0.071633 -4.630238
3.235250e+02 47    -4.838180  0.092733 -4.745447 -4.736106  0.094641 -4.641

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')